In [14]:
%matplotlib inline
import time 
import random
import pickle
import os
import sys
from pathlib import Path
sys.path.append("../src")
sys.path.append("../webapp")

from IPython.display import Audio
from tqdm import tqdm, trange
import pandas as pd
import torch 
import miniaudio

%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Pull user data

In [15]:
DATA_DIR = "/home/keld/projects/music_gen/sc_scrape/data/"
data_obj_path = f"{DATA_DIR}sc_data_v2.pckl"
data = pickle.load(open(data_obj_path, "rb"))

In [16]:
DATA_WEBAPP_DIR = "/n4Ta/sc_webapp_data/"


In [17]:

#

In [4]:
sc_user_data = dict()
sc_user_data = {user: {"followers": followers} for user, followers in data["users_followers_dwl"].items()}

for usr, fans_also_like in data["fans_also_like"].items():
    if usr in sc_user_data:
        valid_links = [usr for usr in fans_also_like if usr in sc_user_data]
        sc_user_data[usr]["fans_also_like"] = valid_links
        for fans_like_usr in valid_links:
            if "users_fans_link_in" not in sc_user_data[fans_like_usr]:
                sc_user_data[fans_like_usr]["users_fans_link_in"] = []
            sc_user_data[fans_like_usr]["users_fans_link_in"].append(usr)
        

Get music associated with user

In [5]:
SONGS_DATA_DF_PATH =  "/n4Ta/sc_webapp_data/" + "df_with_mp3_info.tsv.gz"
all_music_info_df = pd.read_csv(
    SONGS_DATA_DF_PATH,
    compression="gzip",
    sep="\t",
    low_memory=False,
)

In [6]:
len(all_music_info_df)

2198568

In [7]:
all_music_info_df["sc_user"] = all_music_info_df["paths"].apply(lambda x: x.split("/r32Ta/soundcloud_music/files/")[1].split("/")[0])

In [8]:
from genre_conversion import GENRE_RENAME_GROUPS
from collections import Counter

GENRE_RENAME = {}
for genre_to, genre_from_list in GENRE_RENAME_GROUPS.items():
    for genre_from in genre_from_list:
        GENRE_RENAME[genre_from] = genre_to

all_music_info_df["genre"] = all_music_info_df["genre"].apply(lambda x: GENRE_RENAME.get(str(x).lower(), str(x).lower()))

#usr_grouped_df = all_music_info_df.groupby("sc_user")
genre_counter = Counter(all_music_info_df["genre"].values)

most_common = genre_counter.most_common(100)

unmapped_genres = [
    genre for genre, cnt in most_common
    if genre not in GENRE_RENAME_GROUPS
]

unmapped_genre_counts = [
    (genre, cnt) for genre, cnt in most_common
    if genre not in GENRE_RENAME_GROUPS
]
print(unmapped_genres)

['freestyle', 'vocal', 'darktek', 'dream pop', 'dark', 'synthpop', 'americana', 'minimal house', 'axtone', 'trip-hop', 'multicultural electronic', 'umek', 'dance - breaks', 'halloween', 'deep sounds', 'ethereal', 'action', 'baile funk', 'halftime', 'possession', 'bootleg', 'funk de bh', 'various', 'accordion', 'bachata', 'bossa nova', 'early hardcore', 'russ', 'nervous music', 'hardwave', 'funkbh', 'lofi.hiphop', 'league of legends', 'egnu', 'dembow', 'dance - beats & breaks / electronica', 'underground', 'instrumental hip hop', 'selftitledpremiere', 'mozart', 'noise', 'wonderlust', 'dance pop', 'cuff', 'bafta', 'live dj mix', 'album', 'jazz & hip hop', 'electronic: electro', 'korg', 'souletiquette']


In [9]:
GENRE_RENAME_GROUPS = dict(sorted([
    (k, sorted(list(set([g.lower() for g in l]))))
    for k, l in GENRE_RENAME_GROUPS.items()
]))

In [10]:
#GENRE_RENAME_GROUPS

In [11]:
#unmapped_genre_counts

In [12]:
print(
f"""
The following is the list of audio/music/sound genres/classifications currently defined:

{list(GENRE_RENAME_GROUPS.keys())}

List if they the ones that should go into each of the above genres if you are certain in the format:

genre_to: [ 
    < list of genres that should map into it>
]

example:

'Hip-hop': [
    "grime",
    "hip hop",
    "hip hop/rap",
    "hip-hop",
    "hip-hop & rap",
    "hip-hop/rap",
    ]

all the different variations are given in lower case

Only do mapping of the ones you are clear about. Feel free to add things Miscellaneous that are not clear. For the ones that doesn't map. list them and provide reason for why they don't map to any of the genres defined.

<reason>
'< name of missing >'

Here are the unmapped 
{unmapped_genres}
"""
)


The following is the list of audio/music/sound genres/classifications currently defined:

['Acoustic', 'Alternative & Indie Rock', 'Ambient', 'Audiobooks', 'Brazilian', 'Cello', 'Christian/Gospel', 'Christmas/holiday', 'Classical', 'Comedy', 'Contemporary classical', 'Country', 'Disco', 'Drum & bass', 'Educational & Informative', 'Electronic', 'Electronic Dance Music (EDM)', 'Epic', 'Flute', 'Folk', 'Folk/Singer-Songwriter', 'Hip-hop', 'House', 'Indie', 'Instrumental', 'Jazz/Blues', 'Latin', 'Metal', 'Miscellaneous', 'New age', 'News & Politics', 'Opera', 'Piano', 'Podcasts & Talk Shows', 'Pop', 'Psytrance', 'Punk', 'R&B/Soul/Funk', 'Reggae', 'Remix', 'Rock', 'Soundtrack', 'Sports', 'Storytelling', 'Techno', 'Trailer', 'Trance', 'Video game music', 'World', 'asian pop']

List if they the ones that should go into each of the above genres if you are certain in the format:

genre_to: [ 
    < list of genres that should map into it>
]

example:

'Hip-hop': [
    "grime",
    "hip hop",
  

In [13]:
unmapped_genre_counts

[('freestyle', 222),
 ('vocal', 191),
 ('darktek', 191),
 ('dream pop', 188),
 ('dark', 173),
 ('synthpop', 169),
 ('americana', 168),
 ('minimal house', 166),
 ('axtone', 166),
 ('trip-hop', 165),
 ('multicultural electronic', 164),
 ('umek', 163),
 ('dance - breaks', 163),
 ('halloween', 162),
 ('deep sounds', 162),
 ('ethereal', 161),
 ('action', 161),
 ('baile funk', 161),
 ('halftime', 160),
 ('possession', 160),
 ('bootleg', 160),
 ('funk de bh', 160),
 ('various', 159),
 ('accordion', 159),
 ('bachata', 159),
 ('bossa nova', 158),
 ('early hardcore', 158),
 ('russ', 157),
 ('nervous music', 157),
 ('hardwave', 156),
 ('funkbh', 156),
 ('lofi.hiphop', 156),
 ('league of legends', 156),
 ('egnu', 156),
 ('dembow', 155),
 ('dance - beats & breaks / electronica', 155),
 ('underground', 154),
 ('instrumental hip hop', 154),
 ('selftitledpremiere', 154),
 ('mozart', 153),
 ('noise', 153),
 ('wonderlust', 153),
 ('dance pop', 152),
 ('cuff', 152),
 ('bafta', 151),
 ('live dj mix', 151)

## Other

In [2]:
DATA_DIR = "/home/keld/projects/music_gen/sc_scrape/data/"


In [3]:
all_music_info_df = pd.read_csv(
    f"{DATA_DIR}music_info.tsv.gz", 
    compression='gzip', 
    sep='\t', 
    low_memory=False)

In [4]:
all_music_info_df.head()

,album,albumartist,artist,audio_offset,bitrate,channels,composer,disc,disc_total,duration,...,year,local_path,sc_user,sc_track_name,sc_link,tag,likes,reposts,plays,comments
0,NaN,NaN,2CELLOS,86609,128.0,2,NaN,NaN,NaN,400.871937,...,NaN,/d16Tb/soundcloud_music/files/sony-classical/B...,2cellos,2cellos/benedictus,https://soundcloud.com/2cellos/benedictus,classical,5825,690,419957,288
1,NaN,NaN,2CELLOS,83519,128.0,2,NaN,NaN,NaN,204.837345,...,NaN,/d16Tb/soundcloud_music/files/sony-classical/2...,2cellos,2cellos/2cellos---supermassive-black-hole,https://soundcloud.com/2cellos/2cellos---super...,NaN,-1,-1,-1,-1
2,NaN,NaN,2CELLOS,103123,128.0,2,NaN,NaN,NaN,208.497615,...,NaN,/d16Tb/soundcloud_music/files/sony-classical/H...,2cellos,2cellos/highway-to-hell,https://soundcloud.com/2cellos/highway-to-hell,pop,61,1,8359,0
3,NaN,NaN,The Music Showcase,34304,128.0,2,NaN,NaN,NaN,369.681007,...,NaN,/d16Tb/soundcloud_music/files/freemusicstreami...,the music showcase,the music showcase/atlantis-|-paul-landry,https://soundcloud.com/the music showcase/atla...,NaN,-1,-1,-1,-1
4,NaN,NaN,Meditation Music,109594,128.0,2,NaN,NaN,NaN,395.250794,...,NaN,/d16Tb/soundcloud_music/files/freemusicstreami...,meditation music,meditation music/aphrodite-|-paul-landry,https://soundcloud.com/meditation music/aphrod...,NaN,-1,-1,-1,-1


In [5]:
data_obj_path = f"{DATA_DIR}sc_data_v2.pckl"

In [6]:
data = pickle.load(open(data_obj_path, "rb"))

In [7]:
data.keys()

dict_keys(['users_followers_not_dwl', 'users_followers_dwl', 'tracks_stats', 'fans_also_like'])

In [ ]:
#data["tracks_stats"]

In [10]:
#data["fans_also_like"]

In [8]:
len(all_music_info_df)

1443199

In [9]:
! ls {DATA_DIR}

archive_tracker  copy_sc_data_v2.pckl	 music_info.tsv.gz
chromedriver	 music_info_tiny.tsv.gz  sc_data_v2.pckl


In [35]:
def score_terms(x, terms=None):
    if terms:
        terms = [term.lower() for term in terms]
        return sum([str(x).lower().find(term) != -1 for term in terms ])
    else: 
        return True

scores = all_music_info_df["artist"].apply(lambda x: score_terms(x, ['Vienna', "Teng"]))

In [36]:
df_search = all_music_info_df.copy()

df_search["search_score"] = scores
df_search = df_search[df_search["search_score"] != 0]

In [38]:
df_search.search_score

4496       1
9661       1
59672      1
59726      1
59753      1
          ..
1135287    1
1258516    1
1273562    1
1327157    1
1403343    1
Name: search_score, Length: 438, dtype: int64

In [39]:
df_search.head()

,album,albumartist,artist,audio_offset,bitrate,channels,composer,disc,disc_total,duration,...,local_path,sc_user,sc_track_name,sc_link,tag,likes,reposts,plays,comments,search_score
4496,NaN,NaN,Tengo's Trail,88770,128.0,2,NaN,NaN,NaN,155.318852,...,/d16Tb/soundcloud_music/files/schaddenhook/Mor...,tengo's trail,tengo's trail/moramics-part-ii,https://soundcloud.com/tengo's trail/moramics-...,NaN,-1,-1,-1,-1,1
9661,NaN,NaN,Mark Reitenga,62543,128.0,2,NaN,NaN,NaN,214.798530,...,/d16Tb/soundcloud_music/files/schaddenhook/Be ...,mark reitenga,mark reitenga/be-my-friend,https://soundcloud.com/mark reitenga/be-my-friend,NaN,-1,-1,-1,-1,1
59672,Little Big Overtures Box,NaN,Vienna Symphony Orchestra,96438,128.0,2,NaN,NaN,NaN,29.927844,...,/d16Tb/soundcloud_music/files/royal-philharmon...,vienna symphony orchestra,vienna symphony orchestra/franz-joseph-haydn:-...,https://soundcloud.com/vienna symphony orchest...,NaN,-1,-1,-1,-1,1
59726,Little Big Overtures Box,NaN,Vienna State Opera Orchestra,96433,128.0,2,NaN,NaN,NaN,29.927844,...,/d16Tb/soundcloud_music/files/royal-philharmon...,vienna state opera orchestra,vienna state opera orchestra/giocchino-rossini...,https://soundcloud.com/vienna state opera orch...,NaN,-1,-1,-1,-1,1
59753,Little Big Overtures Box,NaN,Vienna State Opera Orchestra,96432,128.0,2,NaN,NaN,NaN,29.927844,...,/d16Tb/soundcloud_music/files/royal-philharmon...,vienna state opera orchestra,vienna state opera orchestra/giocchino-rossini...,https://soundcloud.com/vienna state opera orch...,NaN,-1,-1,-1,-1,1


### Search all music

### Make small df

In [1]:
DATA_WEBAPP_DIR = "/n4Ta/sc_webapp_data/"


In [2]:
info_df_path = DATA_WEBAPP_DIR + "df_with_mp3_info.tsv.gz"


In [5]:
all_music_info_df = pd.read_csv(
    info_df_path,
    compression="gzip",
    sep="\t",
    low_memory=False,
)

In [6]:
len(all_music_info_df)

2198568

In [33]:
all_music_info_df.columns

Index(['album', 'albumartist', 'artist', 'audio_offset', 'bitrate', 'channels',
       'composer', 'disc', 'disc_total', 'duration', 'filesize', 'genre',
       'samplerate', 'title', 'track', 'track_total', 'year', 'local_path',
       'sc_user', 'sc_track_name', 'sc_link', 'tag', 'likes', 'reposts',
       'plays', 'comments'],
      dtype='object')

In [60]:
query = 'test artist: "Vienna Teng"'

In [61]:
valid_categories = {"filename", "title", "artist", "sc_user"}

categories_queries = parse_query_find(query, valid_categories)

In [62]:
categories_queries

{'artist': ('Vienna Teng', True), 'Pre-Category Text': ('test', False)}

In [63]:
df_songs=all_music_info_df

## Create small dataset

In [3]:
DATA_DIR = "/n4Ta/sc_webapp_data/"
info_path = DATA_DIR + "df_with_mp3_info.tsv.gz"
w_info_df = pd.read_csv(info_path, sep="\t", compression="gzip")

In [4]:
len(w_info_df)

2198568

In [5]:
w_info_100k_df = w_info_df.sample(100000)

In [6]:
len(w_info_100k_df), 

(100000,)

In [7]:
w_info_100k_df.to_csv(DATA_DIR + "df_with_mp3_info_100k.tsv.gz", index=False, sep="\t", compression="gzip")

In [17]:
# genre_rename_groups = dict()

# for genre_from, genre_to in GENRE_RENAME.items():
#     if genre_to not in genre_rename_groups:
#         genre_rename_groups[genre_to] = list()
#     genre_rename_groups[genre_to].append(genre_from)


In [18]:

Counter(gr['genre'].values)

NameError: name 'gr' is not defined

In [178]:
#Counter(gr['genre_id'].values)

In [147]:
# load data

In [149]:
w_info_df.columns

Index(['paths', 'filename', 'title', 'artist', 'album', 'album artist',
       'track number', 'release year', 'duration (seconds)', 'bitrate (kbps)',
       'genre', 'genre_id'],
      dtype='object')

In [143]:
sc_user_data

{'pandadub': {'followers': 60800},
 'flume': {'followers': 2210229},
 'diplo': {'followers': 7102848},
 'pushat': {'followers': 8189978, 'users_fans_link_in': ['massappealrecs']},
 'foolsgoldrecs': {'followers': 7925236},
 'maddecent': {'followers': 6014381},
 'skrillex': {'followers': 6504701, 'users_fans_link_in': ['apashe']},
 'dillonfrancis': {'followers': 6135411, 'users_fans_link_in': ['drfresch']},
 'calvinharris': {'followers': 7170101,
  'users_fans_link_in': ['ritontime', 'atb-music']},
 'zedsdead': {'followers': 6012419,
  'users_fans_link_in': ['official-rezz', 'biggigantic', 'ganjawhitenight']},
 'theweeknd': {'followers': 6334323},
 'flosstradamus': {'followers': 5880301,
  'users_fans_link_in': ['etc-etc', 'jayceeoh']},
 'flyinglotus': {'followers': 5825140, 'users_fans_link_in': ['cookinsoul']},
 'a-trak': {'followers': 5806083},
 'bondax': {'followers': 5513684,
  'users_fans_link_in': ['annie-mac-presents',
   'le_marquis',
   'etonmessyrecords']},
 'ninja-tune': {'fo